[<img src="https://colab.research.google.com/assets/colab-badge.svg" />](https://colab.research.google.com/github/RubeRad/tcscs/blob/master/notebooks/30_gps_tracking.ipynb)

# Analyzing GPS tracking data

The New York Times article series, [*One Nation, Tracked*](https://www.nytimes.com/interactive/2019/12/19/opinion/location-tracking-cell-phone.html) showed how easy it is to identified individuals from their smartphone GPS tracks -- and how GPS tracks cannot be "anonymized" simply by omitting names:

<blockquote>In most cases, ascertaining a home location and an office location was enough to identify a person. Consider your daily commute: Would any other smartphone travel directly between your house and your office every day?

Describing location data as anonymous is “a completely false claim” that has been debunked in multiple studies, Paul Ohm, a law professor and privacy researcher at the Georgetown University Law Center, told us. “Really precise, longitudinal geolocation information is absolutely impossible to anonymize.”

“D.N.A.,” he added, “is probably the only thing that’s harder to anonymize than precise geolocation information.”</blockquote>

We're going to do some investigative sleuthing of our own, using some GPS track data (voluntarily submitted by willing participants).  [This full dataset](https://www.microsoft.com/en-us/download/details.aspx?id=52367) from Microsoft Research contains about 25M GPS points with IDs and timestamps, in cities around the world. I've selected a subset in one particular city.

# Looking at the dataset

Load the dataset into a pandas DataFrame, and probe it to get some initial understanding

In [ ]:
import numpy             as np
import matplotlib.pyplot as plt
import pandas            as pd

In [ ]:
# We name this DataFrame dfall, because we're going to be filtering it down later
df = pd.read_csv('https://raw.githubusercontent.com/RubeRad/tcscs/master/notebooks/tracks.csv')
df

In [ ]:
df.describe()

In [ ]:
# LONgitude =  East/West  = X 
# LATitude  = North/South = Y
plt.plot(df.LON, df.LAT)

## Exercise

* How much data is there?
* What time span does the data cover?
* Since 1deg$\approx$100km (thus 1m$\approx$0.00001deg), what geographic extent is the dataset?
* What city is this?

As loaded, the LAT, LON columns are numerical. This creates a new column called 'LATLON' which smushes them together as strings with 'N' and 'W'. We'll use this in a minute...

In [ ]:
df['LATLON'] = df.LAT.astype(str) + 'N ' + (abs(df.LON)).astype(str) + 'W'
df

# Plotting the individuals separately

All that mess plotted above is as if the GPS tracks came from just one person. We can use the ID to separate per-person track data

In [ ]:
df.ID.value_counts()

Let's practice row-slicing, so we can plot these tracks separately

In [ ]:
# make a slice of rows of the most common ID

In [ ]:
# Plot it

In [ ]:
# make a slice of rows of the 2nd most common ID

In [ ]:
# Make a plot with the common and 2nd most common both plotted

How many of these are we going to have to do?

In [ ]:
df.ID.unique()

In [ ]:
ids = df.ID.unique()
len(ids)

# Plotting all the tracks -- LET'S GET LAZY!

Now that we have a list of all the unique IDs, we can make a loop that filters the DataFrame down to just one ID at a time, and plot them separately.

In [ ]:
# start with matplotlib boilerplate

for id in ids:
    print(id)  # you can eventually delete this
    # make a row-slice for this id
    # ax.plot() the LON, LAT of that slice
    

## Exercise

Add the slicing and plotting as suggested above, and then do these things to make this plot more readable:

* Adjust the `figsize`
* `ax.set_aspect('equal')` (because it's a map, we don't want it squished)
* Use `str(id)` as the label for each plot, and turn on `ax.legend()` at the end
* `ax.set_xlim(  ,  )`     (make room for the legend)
* Add the index to each legend label
  * Change the item-only loop to a combo index/item loop using `enumerate(ids)`
  * Build a more sophisticated label using the index and the id

# Inspect a particular track

The map above of everybody's tracks might help choose a particular track to look at in isolation. Let's just start with the first one.

In [ ]:
id = ids[0]
id

In [ ]:
dfid = df[ dfall.ID==id ]
dfid

In [ ]:
fig = plt.figure(figsize=(12,12))
ax  = fig.add_subplot()

# this is so LAT/LON are proportional like a map
#ax.set_aspect('equal')
# this makes sure matplotlib doesn't put the X axis in scientific notation
#ax.ticklabel_format(useOffset=False, style='plain')
# this rotates the longitudes so they don't write on top of each other
#plt.setp(ax.get_xticklabels(), rotation=30, horizontalalignment='right')

ax.plot(dfid.LON, dfid.LAT)

In [ ]:
dfid.LAT.describe()

In [ ]:
dfid.LON.describe()

## Exercise

* Look at the beginning/end of `dfid`
  * When/where does this person start/end their day (track)?
  * About how far apart are the start/end points? (1deg$\approx$100km; 1m$\approx$0.00001deg)
  * About how much time is there between the start/end points? 
  * What's the time resolution of this data?
* Look at the plot
  * Where are the start/end points?
  * About how big is the total extent of the track?
* Go to Google Maps, and plot directions from the first LATLON to the last LATLON


## Identifying extreme points of the track

What's that spike on the West side? `dfid.LON.describe()` says the minimum LON is -122.143624

In [ ]:
# It might work to filter for ==-122.143624 exactly, 
# but just in case of rounding error...
dfid[ dfid.LON<-122.14362 ]

Where's that southernmost bump?

In [ ]:
dfid[ dfid.LAT<47.62793 ]

## Exercise

* Add those two LATLON to the Google Map directions
* Check the DATETIME, make sure they are in the right order
* How far apart are those two trackpoints in space?
* How far apart are those two trackpoints in time?
* What is the likely mode of travel?

## Identifying other parts of the track

Let's pull out that knob at the NW corner, as distinct from the two other knobs at that maximum latitude, which are further east.

In [ ]:
# parentheses are critically important here
knob = dfid[ (dfid.LAT>47.6325) & (dfid.LON>-122.1425) & (dfid.LON<-122.1415) ]
knob

In [ ]:
knob.describe()

In [ ]:
knob[ knob.LAT > 47.63289 ]

## Exercise

* Paste this LATLON also into the Google Map directions, in proper sequence

# Homework

Select a different ID, and repeat the exercises above

* Create a map of their track
* Identify the start and end points
* Choose and isolate at least two other points of interest from somewhere in the middle of their track
* Build up GoogleMap directions for the start/selected/end points -- in chronological order
* Conclude with one or more markup cells that escribe whatever kind of inferences we can make from this data:
  * What this person was doing?
  * Where do they live? work? shop? eat? recreate?
  * How fast are they moving? What is their mode of transport?
  * etc
  * Include a link to the URL of the GoogleMap directions